In [ ]:
from datetime import datetime, timezone
import os
import dotenv
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)

# Load environment variables
dotenv.load_dotenv()

# Constants
MAX_RETRIES = int(os.environ.get("MAX_RETRIES", "3"))
RETRY_DELAY = int(os.environ.get("RETRY_DELAY", "5"))
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
DEPLOYMENT_NAME = os.getenv("DEPLOYMENT_NAME")

# Model settings
TEMPERATURE = float(os.getenv("TEMPERATURE", 0.2))
TOP_P = float(os.getenv("TOP_P", 0.2))
MODEL_NAME = os.getenv("MODEL_NAME", "gpt-4o")


In [ ]:
import openai

def call_llm_service(prompt, response_format="text"):
    client = openai.AzureOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_KEY,
        api_version=AZURE_OPENAI_API_VERSION,
    )

    messages = [{"role": "system", "content": prompt}]
    if response_format == "json":
        messages[0]["content"] += "\nPlease provide the response strictly in JSON format."

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        max_tokens=4096,
        temperature=TEMPERATURE,
        top_p=TOP_P
    )

    return response.choices[0].message.content.strip()


In [ ]:
import fitz  # PyMuPDF

def load_text_file(filepath, fallback=""):
    try:
        with open(filepath, "r", encoding="utf-8") as file:
            return file.read()
    except Exception as e:
        logging.error(f"Error loading {filepath}: {e}")
        return fallback

def extract_pdf_file(filepath):
    try:
        with fitz.open(filepath) as pdf:
            return "\n".join(page.get_text() for page in pdf)
    except Exception as e:
        logging.error(f"Error reading PDF {filepath}: {e}")
        return ""

In [ ]:
# Paths to text and PDF files (edit as needed)
context_path = "path/to/context_prompt.txt"
glossary_path = "path/to/glossary.txt"
user_story_path = "path/to/user_story.txt"
business_doc_path = "path/to/business_doc.pdf"
solution_doc_path = "path/to/solution_doc.pdf"
generator_prompt_path = "path/to/generator_prompt.txt"

# Load each component
context = load_text_file(context_path).strip()
glossary = load_text_file(glossary_path).strip()
user_story = load_text_file(user_story_path).strip()
business_doc = extract_pdf_file(business_doc_path).strip()

In [ ]:
all_support_documents_with_name_tags = (
    "# Business Document\n" + business_doc + "\n\n" +
    "# User Story\n" + user_story
)

In [ ]:
generator_prompt_template = load_text_file(generator_prompt_path).strip()

# Fill the prompt template safely
filled_generator_prompt = generator_prompt_template.format(
    context=context,
    glossary=glossary,
    user_story=user_story,
    all_support_documents_with_name_tags=all_support_documents_with_name_tags
)

In [ ]:
generator_output = call_llm_service(filled_generator_prompt)
print("Generator Output:\n")
print(generator_output)